In [3]:
from pyspark import SparkContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [4]:
sc = SparkContext()

In [34]:
#Select the desired columns
data = sc.textFile('./lastfm-dataset-360k-small/merged-subset.csv').map(lambda x : [x.split(',')[i] for i in [1,2,4]])
header = data.take(1)[0]

In [35]:
#Remove header
data2 = data.filter(lambda line: line!=header)
data2.take(2)

[['0aa7f82c5507f4a1f84240775eb4f2d26e558a9e',
  '54b26205-3b93-44f9-9f91-36dba8330f6d',
  '44'],
 ['528c991795ffd31e11db80a7d299d2438466b52b',
  '4a4ee089-93b1-4470-af9a-6ff575d32704',
  '156']]

In [36]:
#Convert strings into integers
users = data2.map(lambda x: x[0]).distinct().zipWithIndex()
artists = data2.map(lambda x: x[1]).distinct().zipWithIndex()
# int_user = users.map(lambda u: (u[1], u[0]))
# int_artist = artists.map(lambda i: (i[1], i[0]))

In [37]:
# Substitutes the ObjectIDs in the ratings RDD with the corresponding int values
data2 = data2.map(lambda r: (r[0], (r[1], r[2]))).join(users).map(lambda r: (r[1][1], r[1][0][0], r[1][0][1]))
data2 = data2.map(lambda r: (r[1], (r[0], r[2]))).join(artists).map(lambda r: (r[1][0][0], r[1][1], r[1][0][1]))

In [38]:
data2.first()

(12, 11, '168')

In [39]:
data2.take(5)

[(12, 11, '168'),
 (18, 23, '178'),
 (27, 30, '17'),
 (6232, 30, '38'),
 (33, 33, '59')]